In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## 1. Định nghĩa lớp Neural Network

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_stack(x)
        return logits

## 2. Chuyển mô hình sang device để chuẩn bị huấn luyện

In [6]:
model = NeuralNetwork().to(device)
print(f"model: \n{model}\n")

model: 
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)



## 3. Dự đoán giá trị đầu ra

In [7]:
X = torch.randn(1, 28,28,device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"y_pred: {y_pred}")

y_pred: tensor([6], device='cuda:0')


## 4. Phân tích các Layers

In [13]:
#input 3 ảnh kích thước 28x28
input_image = torch.rand(3,28,28)
print(f"input_image: {input_image.size()}\n")

input_image: torch.Size([3, 28, 28])



In [12]:
#Lớp Flatten chuyển về vector phẳng 3 x 28x28 = 784
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [14]:
#Lớp tuyến tính sử dụng biến đổi tuyến tính trên đầu vào với trọng số và bias
layer1 = nn.Linear(in_features=28*28,out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [15]:
# Hàm kích hoạt phi tuyến ReLU dùng để ánh xạ input và output của model. Thực hiện biến đổi phi tuyến bằng hàm kích hoạt ReLU
print(f"Before ReLU: {hidden1}\n ")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}\n")

Before ReLU: tensor([[ 3.1865e-02, -2.4625e-01, -2.6016e-02, -2.3709e-01, -6.2701e-02,
         -1.6343e-01, -1.4148e-01,  2.7189e-02,  7.6988e-04,  3.7723e-02,
          6.4863e-02,  1.1575e-01,  5.2005e-01, -5.0643e-02, -4.0064e-01,
          2.8441e-01, -2.5574e-01,  8.2022e-02,  8.8556e-02,  1.4547e-01],
        [ 1.7520e-01, -2.7322e-01, -2.8195e-01, -9.2384e-02,  4.9458e-02,
         -1.7391e-02, -3.3538e-01,  7.6093e-02, -3.4667e-04, -1.2512e-01,
          6.3781e-02,  3.9614e-02,  6.5756e-01, -1.9252e-02, -4.2553e-01,
          3.5806e-01, -1.3238e-01,  4.3512e-02,  1.2835e-02,  1.1239e-01],
        [ 6.4933e-01, -1.2304e-01, -1.8138e-01, -1.8567e-01,  9.1221e-02,
          2.3643e-02,  2.4345e-03,  3.0192e-01, -6.0268e-02, -1.0704e-01,
          1.0123e-01, -1.4673e-01,  6.7450e-01, -1.7930e-01, -4.9973e-01,
          3.2232e-01, -6.8332e-02, -1.7475e-01, -5.1150e-02,  5.7678e-01]],
       grad_fn=<AddmmBackward0>)
 
After ReLU: tensor([[0.0319, 0.0000, 0.0000, 0.0000, 0.0000,

In [17]:
# Sequential tạo một container để chứa các modul layers
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(in_features=20,out_features=10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [18]:
# nn.Softmax lớp tuyến tính đầu ra trả về logits dùng để phân loại xác suất đầu ra trong khoảng [0,1]
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [19]:
# Hiện các tham số của mô hình
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0216,  0.0139,  0.0165,  ..., -0.0041, -0.0321,  0.0197],
        [ 0.0125,  0.0029, -0.0160,  ...,  0.0351, -0.0223,  0.0336]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0009, 0.0318], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0416,  0.0064, -0.0263,  ...,  0.0040,  0.0242,  0.0192],
        [-0.0043, -0.0137,  0.0384,  ...,  0.0199, -0.0392,  0.0161]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer